In [1]:
import os
import numpy as np
from collections import OrderedDict

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
from tflite import Model

data = open("./mediapipe/mediapipe/models/iris_landmark.tflite", "rb").read()
model = Model.Model.GetRootAsModel(data, 0)

In [3]:
subgraph = model.Subgraphs(0)
subgraph.Name()

b'faceeyenbrow_iris_full_2019_09_13_v0'

In [4]:
def get_shape(tensor):
    return [tensor.Shape(i) for i in range(tensor.ShapeLength())]

In [5]:
for i in range(0, subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    print("%3d %30s %d %2d %s" % (i, tensor.Name(), tensor.Type(), tensor.Buffer(), 
                                  get_shape(subgraph.Tensors(i))))

  0                     b'input_1' 0  0 [1, 64, 64, 3]
  1               b'conv2d/Kernel' 0  1 [64, 3, 3, 3]
  2                 b'conv2d/Bias' 0  2 [64]
  3                      b'conv2d' 0  0 [1, 32, 32, 64]
  4               b'p_re_lu/Alpha' 0  3 [1, 1, 64]
  5                     b'p_re_lu' 0  0 [1, 32, 32, 64]
  6             b'conv2d_1/Kernel' 0  4 [32, 1, 1, 64]
  7               b'conv2d_1/Bias' 0  5 [32]
  8                    b'conv2d_1' 0  0 [1, 32, 32, 32]
  9             b'p_re_lu_1/Alpha' 0  6 [1, 1, 32]
 10                   b'p_re_lu_1' 0  0 [1, 32, 32, 32]
 11     b'depthwise_conv2d/Kernel' 0  7 [1, 3, 3, 32]
 12       b'depthwise_conv2d/Bias' 0  8 [32]
 13            b'depthwise_conv2d' 0  0 [1, 32, 32, 32]
 14             b'conv2d_2/Kernel' 0  9 [64, 1, 1, 32]
 15               b'conv2d_2/Bias' 0 10 [64]
 16                    b'conv2d_2' 0  0 [1, 32, 32, 64]
 17         b'add__xeno_compat__1' 0  0 [1, 32, 32, 64]
 18             b'p_re_lu_2/Alpha' 0 11 [1, 1, 64]
 1

In [6]:
tensor_dict = {(subgraph.Tensors(i).Name().decode("utf8")): i 
               for i in range(subgraph.TensorsLength())}

In [7]:
parameters = {}
for i in range(subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    if tensor.Buffer() > 0:
        name = tensor.Name().decode("utf8")
        parameters[name] = tensor.Buffer()

len(parameters)

216

In [8]:
def get_weights(tensor_name):
    i = tensor_dict[tensor_name]
    tensor = subgraph.Tensors(i)
    buffer = tensor.Buffer()
    shape = get_shape(tensor)
    assert(tensor.Type() == 0)  # FLOAT32
    
    W = model.Buffers(buffer).DataAsNumpy()
    W = W.view(dtype=np.float32)
    W = W.reshape(shape)
    return W

In [9]:
W = get_weights("conv2d/Kernel")
b = get_weights("conv2d/Bias")
W.shape, b.shape

((64, 3, 3, 3), (64,))

In [10]:
import torch
from blazeiris import Blazeiris

In [11]:
net = Blazeiris()
net

Blazeiris(
  (backbone1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): PReLU(num_parameters=64)
    (2): Blazeblock1(
      (convs): Sequential(
        (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): PReLU(num_parameters=32)
        (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
        (3): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
      )
      (act): PReLU(num_parameters=64)
    )
    (3): Blazeblock1(
      (convs): Sequential(
        (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): PReLU(num_parameters=32)
        (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
        (3): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
      )
      (act): PReLU(num_parameters=64)
    )
    (4): Blazeblock1(
      (convs): Sequential(
        (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): PReLU(num_parameter

In [12]:
import re

probable_names = []
tmp = []
for i in range(0, subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    if i < 149 and tensor.Buffer() > 0 and tensor.Type() == 0:
        probable_names.append(tensor.Name().decode("utf-8"))
    if i > 149 and i < 377 and tensor.Buffer() > 0 and tensor.Type() == 0:
        if tensor.Name().decode("utf-8")[0] != 'd':
            if int(re.sub("\D", "", tensor.Name().decode("utf-8"))) % 100 < 37:
                probable_names.append(tensor.Name().decode("utf-8"))
            else: tmp.append(tensor.Name().decode("utf-8"))
        else:
            if int(re.sub("\D", "", tensor.Name().decode("utf-8"))) % 100 < 18:
                probable_names.append(tensor.Name().decode("utf-8"))
            else: tmp.append(tensor.Name().decode("utf-8"))
                
                
tensor = subgraph.Tensors(378)
probable_names.append(tensor.Name().decode("utf-8"))
tensor = subgraph.Tensors(379)
probable_names.append(tensor.Name().decode("utf-8"))

tensor = subgraph.Tensors(381)
tmp.append(tensor.Name().decode("utf-8")) 
tensor = subgraph.Tensors(382)
tmp.append(tensor.Name().decode("utf-8"))

probable_names = probable_names + tmp

for i in range(len(probable_names)):
    print(probable_names[i])        

conv2d/Kernel
conv2d/Bias
p_re_lu/Alpha
conv2d_1/Kernel
conv2d_1/Bias
p_re_lu_1/Alpha
depthwise_conv2d/Kernel
depthwise_conv2d/Bias
conv2d_2/Kernel
conv2d_2/Bias
p_re_lu_2/Alpha
conv2d_3/Kernel
conv2d_3/Bias
p_re_lu_3/Alpha
depthwise_conv2d_1/Kernel
depthwise_conv2d_1/Bias
conv2d_4/Kernel
conv2d_4/Bias
p_re_lu_4/Alpha
conv2d_5/Kernel
conv2d_5/Bias
p_re_lu_5/Alpha
depthwise_conv2d_2/Kernel
depthwise_conv2d_2/Bias
conv2d_6/Kernel
conv2d_6/Bias
p_re_lu_6/Alpha
conv2d_7/Kernel
conv2d_7/Bias
p_re_lu_7/Alpha
depthwise_conv2d_3/Kernel
depthwise_conv2d_3/Bias
conv2d_8/Kernel
conv2d_8/Bias
p_re_lu_8/Alpha
conv2d_9/Kernel
conv2d_9/Bias
p_re_lu_9/Alpha
depthwise_conv2d_4/Kernel
depthwise_conv2d_4/Bias
conv2d_10/Kernel
conv2d_10/Bias
p_re_lu_10/Alpha
conv2d_11/Kernel
conv2d_11/Bias
p_re_lu_11/Alpha
depthwise_conv2d_5/Kernel
depthwise_conv2d_5/Bias
conv2d_12/Kernel
conv2d_12/Bias
p_re_lu_12/Alpha
conv2d_13/Kernel
conv2d_13/Bias
p_re_lu_13/Alpha
depthwise_conv2d_6/Kernel
depthwise_conv2d_6/Bias
conv

In [13]:
convert = {}
i = 0
for name, params in net.state_dict().items():
    convert[name] = probable_names[i]
    i += 1

In [14]:
new_state_dict = OrderedDict()

for dst, src in convert.items():
    W = get_weights(src)
    print(dst, src, W.shape, net.state_dict()[dst].shape)

    if W.ndim == 4:       
        if W.shape[0] == 1:
            W = W.transpose((3, 0, 1, 2))  # depthwise conv
        else:
            W = W.transpose((0, 3, 1, 2))  # regular conv        
            
    if W.ndim == 3:
        W = W.reshape(W.shape[2]) #PReLU 
        
    
    
    new_state_dict[dst] = torch.from_numpy(W)

backbone1.0.weight conv2d/Kernel (64, 3, 3, 3) torch.Size([64, 3, 3, 3])
backbone1.0.bias conv2d/Bias (64,) torch.Size([64])
backbone1.1.weight p_re_lu/Alpha (1, 1, 64) torch.Size([64])
backbone1.2.convs.0.weight conv2d_1/Kernel (32, 1, 1, 64) torch.Size([32, 64, 1, 1])
backbone1.2.convs.0.bias conv2d_1/Bias (32,) torch.Size([32])
backbone1.2.convs.1.weight p_re_lu_1/Alpha (1, 1, 32) torch.Size([32])
backbone1.2.convs.2.weight depthwise_conv2d/Kernel (1, 3, 3, 32) torch.Size([32, 1, 3, 3])
backbone1.2.convs.2.bias depthwise_conv2d/Bias (32,) torch.Size([32])
backbone1.2.convs.3.weight conv2d_2/Kernel (64, 1, 1, 32) torch.Size([64, 32, 1, 1])
backbone1.2.convs.3.bias conv2d_2/Bias (64,) torch.Size([64])
backbone1.2.act.weight p_re_lu_2/Alpha (1, 1, 64) torch.Size([64])
backbone1.3.convs.0.weight conv2d_3/Kernel (32, 1, 1, 64) torch.Size([32, 64, 1, 1])
backbone1.3.convs.0.bias conv2d_3/Bias (32,) torch.Size([32])
backbone1.3.convs.1.weight p_re_lu_3/Alpha (1, 1, 32) torch.Size([32])
bac

backbone2.8.bias conv_eyes_contours_and_brows/Bias (213,) torch.Size([213])
backbone3.0.convs.0.weight conv2d_37/Kernel (64, 1, 1, 128) torch.Size([64, 128, 1, 1])
backbone3.0.convs.0.bias conv2d_37/Bias (64,) torch.Size([64])
backbone3.0.convs.1.weight p_re_lu_37/Alpha (1, 1, 64) torch.Size([64])
backbone3.0.convs.2.weight depthwise_conv2d_18/Kernel (1, 3, 3, 64) torch.Size([64, 1, 3, 3])
backbone3.0.convs.2.bias depthwise_conv2d_18/Bias (64,) torch.Size([64])
backbone3.0.convs.3.weight conv2d_38/Kernel (128, 1, 1, 64) torch.Size([128, 64, 1, 1])
backbone3.0.convs.3.bias conv2d_38/Bias (128,) torch.Size([128])
backbone3.0.act.weight p_re_lu_38/Alpha (1, 1, 128) torch.Size([128])
backbone3.1.convs.0.weight conv2d_39/Kernel (64, 1, 1, 128) torch.Size([64, 128, 1, 1])
backbone3.1.convs.0.bias conv2d_39/Bias (64,) torch.Size([64])
backbone3.1.convs.1.weight p_re_lu_39/Alpha (1, 1, 64) torch.Size([64])
backbone3.1.convs.2.weight depthwise_conv2d_19/Kernel (1, 3, 3, 64) torch.Size([64, 1, 3

In [15]:
net.load_state_dict(new_state_dict, strict=True)

<All keys matched successfully>

In [16]:
torch.save(net.state_dict(), "blazeiris.pth")